# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [11]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        # result.append(n ** 0.5)
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [12]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [48]:
#Install the dependencies
# %%capture
!pip install dlt[duckdb]

     ---------------------------------------- 11.6/11.6 MB 3.1 MB/s eta 0:00:00


You should consider upgrading via the 'C:\Users\Asus\Code\data-engineering-zoomcamp\workshops\01-data-ingestion\env\Scripts\python.exe -m pip install --upgrade pip' command.


In [45]:
# to do: homework :)
import dlt
import duckdb
import numpy

pipeline = dlt.pipeline(destination='duckdb', dataset_name='person')


## Question 1: **What is the sum of the outputs of the generator for limit = 5?**

In [17]:
limit = 5
generator = square_root_generator(limit)

total_sum = sum(generator)
print(total_sum)

8.382332347441762


## Question 2: **What is the 13th number yielded**

In [26]:
limit = 13
generator = square_root_generator(limit)

counter = 1
for sqrt_value in generator:
    if counter == limit:
        print(f"the {counter}th value is", sqrt_value)
    else:
        counter += 1
        continue

the 13th value is 3.605551275463989


# Question 3: Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**

In [50]:
def pipeline_to_db(name_of_pipeline, table_name):
    # connect to db
    conn = duckdb.connect(f"{name_of_pipeline.pipeline_name}.duckdb")

    # display table
    conn.sql(f"SET search_path = '{name_of_pipeline.dataset_name}'")
    print("Loaded Tables: ")
    display(conn.sql("show tables"))

    # display data
    return conn.sql(f"Select * from {table_name}").df()


In [74]:
pipeline.run(people_1(), table_name="people_1", write_disposition="replace")
pipeline.run(people_2(), table_name="people_2", write_disposition="replace")

df1 = pipeline_to_db(name_of_pipeline=pipeline, table_name='people_1')
df2 = pipeline_to_db(name_of_pipeline=pipeline, table_name='people_2')

Loaded Tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_1            │
│ people_2            │
└─────────────────────┘

Loaded Tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_1            │
│ people_2            │
└─────────────────────┘

In [77]:
df1.age.sum() + df2.age.sum()

353

## Question 4: **Merge the 2 generators using the ID column. Calculate the sum of ages of all the people loaded as described above.**

In [86]:
final_df = df1.merge(df2, on = 'id')
print(final_df)
sum(final_df['age_x'] + final_df['age_y'])

   id    name_x  age_x  city_x     _dlt_load_id_x       _dlt_id_x    name_y  \
0   3  Person_3     28  City_A  1714699365.121332  dbGE/XDFJqR6Mw  Person_3   
1   4  Person_4     29  City_A  1714699365.121332  /CYlnIRbBvBWXw  Person_4   
2   5  Person_5     30  City_A  1714699365.121332  hS13ohmF5//rYA  Person_5   

   age_y  city_y occupation     _dlt_load_id_y       _dlt_id_y  
0     33  City_B      Job_3  1714699366.104134  8TO+6QnigwZ79g  
1     34  City_B      Job_4  1714699366.104134  wrCQqnHkbjuHzA  
2     35  City_B      Job_5  1714699366.104134  DlpGRxvw+x4opw  


189

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX